In [1]:
import onnxruntime as ort
import numpy as np
import pandas as pd

def load_onnx_model(onnx_file_path):
    """
    Loads a saved ONNX model from the given file path.

    Parameters:
    - onnx_file_path: str - Path to the ONNX model file.

    Returns:
    - onnxruntime.InferenceSession - Loaded ONNX model.
    """
    try:
        session = ort.InferenceSession(onnx_file_path)
        print(f"ONNX model loaded successfully from {onnx_file_path}")
        return session
    except Exception as e:
        print(f"Failed to load ONNX model from {onnx_file_path}: {e}")
        return None

def predict_with_onnx(session, X):
    """
    Runs predictions using the loaded ONNX model.

    Parameters:
    - session: onnxruntime.InferenceSession - Loaded ONNX model session.
    - X: pd.DataFrame or np.array - Input features for prediction.

    Returns:
    - np.array - Model predictions.
    """
    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()  # Convert DataFrame to numpy array
    
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    
    preds = session.run([output_name], {input_name: X.astype(np.float32)})[0]
    return preds

In [4]:
from single_cat_model import SingleCategoryModel

model_placeholder = SingleCategoryModel(category_number=1)

# Load the ONNX model
onnx_model_path = "./models/onnx/category_1_model.onnx"
onnx_session = load_onnx_model(onnx_model_path)

# prepare the data
df = pd.read_json('../data/dataset.json').drop(columns=['sold_price'])
df = model_placeholder.preprocess_data(df=df)

df = df.drop(columns=['sold_price'], errors='ignore')

df.head()

Initialized SingleCategoryModel for category 1.
ONNX model loaded successfully from ./models/onnx/category_1_model.onnx


,price,view_count,steam_item_id,steam_cs2_profile_rank,steam_level,steam_friend_count,steam_dota2_solo_mmr,steam_converted_balance,steam_pubg_inv_value,steam_cs2_inv_value,...,steam_cs2_last_activity_is_weekend_0,steam_cs2_last_activity_is_weekend_1,steam_cs2_ban_date_is_weekend_0,steam_cs2_ban_date_is_weekend_1,steam_last_transaction_date_is_weekend_0,steam_last_transaction_date_is_weekend_1,steam_market_ban_end_date_is_weekend_0,steam_market_ban_end_date_is_weekend_1,steam_cs2_last_launched_is_weekend_0,steam_cs2_last_launched_is_weekend_1
0,597,124,70108402,4,10,23,0,0,-1,130,...,True,False,True,False,True,False,True,False,True,False
1,75,17,71126523,2,0,0,0,0,-1,243,...,True,False,True,False,True,False,True,False,True,False
2,1000,92,69083816,17,20,27,0,8,-1,18,...,True,False,True,False,True,False,True,False,True,False
3,200,17,70902299,0,4,1,0,0,-1,0,...,True,False,True,False,False,True,True,False,True,False
4,120,25,71066650,3,2,3,0,0,-1,90,...,True,False,True,False,True,False,True,False,True,False


In [5]:
# Predict using the ONNX model
if onnx_session is not None:
    predictions = predict_with_onnx(onnx_session, df)
    print("Predictions:", predictions)

2024-11-25 16:31:39.316231 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1} does not match actual shape of {129624,1} for output predictions


Predictions: [[575.0677  ]
 [ 79.10138 ]
 [984.645   ]
 ...
 [449.80392 ]
 [  5.103546]
 [393.7421  ]]
